In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
     .master("local[*]") \
     .appName("sql_spark") \
     .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/04 22:08:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Read parquet files into dataframes

In [2]:
df_green = spark.read.parquet("data/pq/green/*/*")

In [5]:
df_green.explain()

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [VendorID#0,lpep_pickup_datetime#1,lpep_dropoff_datetime#2,store_and_fwd_flag#3,RatecodeID#4,PULocationID#5,DOLocationID#6,passenger_count#7,trip_distance#8,fare_amount#9,extra#10,mta_tax#11,tip_amount#12,tolls_amount#13,ehail_fee#14,improvement_surcharge#15,total_amount#16,payment_type#17,trip_type#18,congestion_surcharge#19] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(19 paths)[file:/home/xiangivyli/data_engineering_zoomcamp_learning/05-batch-pys..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<VendorID:int,lpep_pickup_datetime:timestamp,lpep_dropoff_datetime:timestamp,store_and_fwd_...


None


In [3]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-23 13:10:15|  2020-01-23 13:38:16|                 N|         1|          74|         130|              1|        12.77|       36.0|  0.0|    0.

In [3]:
df_yellow = spark.read.parquet("data/pq/yellow/*/*")

In [14]:
df_green.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [15]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



## Combine these two files into one

In [22]:
# check columns
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [23]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

adjust datetime name

In [4]:
df_green = df_green \
  .withColumnRenamed("lpep_pickup_datetime", "pickup_datetime") \
  .withColumnRenamed("lpep_dropoff_datetime", "dropoff_datetime")

In [5]:
df_yellow = df_yellow \
  .withColumnRenamed("tpep_pickup_datetime", "pickup_datetime") \
  .withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")

In [6]:
set(df_green.columns) & set(df_yellow.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

Another method to find the union set and still keep the sequence

In [7]:
common_columns = []

yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_columns.append(col)

In [8]:
common_columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

add one column to classify yellow and green before merging them together

In [9]:
from pyspark.sql import functions as F

In [10]:
df_green_sel = df_green \
  .select(common_columns)  \
  .withColumn("service_type", F.lit("green"))

In [11]:
df_yellow_sel = df_yellow \
  .select(common_columns)  \
  .withColumn("service_type", F.lit("yellow"))

combine tables together

In [12]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [13]:
df_trips_data.groupBy("service_type").count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



## Querying with SQL

In [38]:
df_trips_data.registerTempTable("trips_data")

/home/xiangivyli/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [40]:
spark.sql("""
SELECT 
    SERVICE_TYPE,
    COUNT(1)
FROM 
    trips_data
GROUP BY 
service_type
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [46]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc("month", "pickup_datetime") AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,

    -- Additional calculations
    AVG(passenger_count) AS avg_monthly_passenger_count,
    AVG(trip_distance) AS avg_monthly_trip_distance

FROM 
    trips_data
GROUP BY 
    revenue_zone, revenue_month, service_type
""")

In [51]:
df_result.coalesce(1).write.parquet("data/report/revenue/", mode="overwrite")

In [52]:
df_result.show()

+------------+-------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+---------------------------+-------------------------+
|revenue_zone|revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|avg_monthly_passenger_count|avg_monthly_trip_distance|
+------------+-------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+---------------------------+-------------------------+
|         254|         null|       green|   278150.7100000017|   11320.520000000006|                  854.5|         9569.900000000001|       